# RAG Testing Notebook

This notebook tests the RAG system with different queries, cleaning output to show only question, answer and useful medatada to understand internal routing

In [1]:
import os
import sys
from io import StringIO

# Disable warnings on macOS
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'
os.environ['GRPC_TRACE'] = ''

from rag import RAGWrapper

# Initialize RAG wrapper
rag = RAGWrapper()

/Users/francesco/Desktop/ACCA/rag/.venv/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
def test_query(question: str):
    
    # Capture stdout to filter it
    old_stdout = sys.stdout
    captured_output = StringIO()
    sys.stdout = captured_output
    
    try:
        response = rag.ask(question, clean_response=True)
    finally:
        # Restore stdout
        sys.stdout = old_stdout
    
    # Get captured output
    output = captured_output.getvalue()
    
    # Print routing information
    for line in output.split('\n'):
        if 'Routing query to files:' in line or 'Filtering by titles:' in line:
            print(line)
    
    # Print clean Q&A
    print(f"\nDomanda: {question}")
    print(f"\nRisposta: {response['answer']}")

## Test Queries

In [3]:
test_query("Quali sono i doveri del lavoratore?")


Domanda: Quali sono i doveri del lavoratore?

Risposta: **Doveri del lavoratore**

1. **Diligenza nell’esecuzione della prestazione**  
   - Il lavoratore deve svolgere la propria attività con la diligenza richiesta dalla natura della prestazione e dall’interesse dell’impresa, rispettando i criteri qualitativi e quantitativi stabiliti.  
   - La mancanza di diligenza può comportare sanzioni disciplinari, fino al licenziamento, e l’obbligo di risarcire l’azienda per eventuali danni contrattuali.

2. **Osservanza delle norme di sicurezza e salute**  
   - È tenuto a rispettare le disposizioni del D.Lgs. 9 aprile 2008 n. 81, a cui il datore di lavoro (o l’utilizzatore, nel caso di somministrazione) fornisce informazione, formazione e addestramento sull’uso delle attrezzature.  
   - Deve collaborare con le misure di prevenzione e protezione adottate dall’azienda.

3. **Obbligo di informazione e comunicazione**  
   - Quando il lavoratore è adibito a mansioni diverse da quelle contrattual

### File and Title routing

In [20]:
test_query("Cosa dice l'articolo 23 del ccnl?")

==================Routing query to files: ccnl_commercio_terziario_distribuzione_e_servizi.pdf
Filtering by titles: Art.23 - Conservazione del posto di lavoro

Domanda: Cosa dice l'articolo 23 del ccnl?

Risposta: L’articolo 23 del CCNL prevede che, durante il periodo di congedo, il lavoratore mantenga il proprio posto di lavoro, ma non abbia diritto alla retribuzione né alla decorrenza dell’anzianità per alcun istituto. Inoltre, è vietato svolgere qualsiasi attività lavorativa durante tale periodo.


### File and routing and itelligent Title routing

In [4]:
test_query("Cosa comporta un passaggio di livello in azienda secondo il ccnl?")

==================Routing query to files: ccnl_commercio_terziario_distribuzione_e_servizi.pdf
Filtering by titles: Art.3 - Mansioni lavorative e passaggi di livello

Domanda: Cosa comporta un passaggio di livello in azienda secondo il ccnl?

Risposta: Un passaggio di livello, così come disciplinato dal CCNL di riferimento, comporta le seguenti conseguenze per il lavoratore:

1. **Retribuzione contrattuale**  
   - Il dipendente ha diritto a percepire la retribuzione prevista dal contratto collettivo per il nuovo livello di inquadramento.  
   - Qualora il dipendente percepisca già elementi retributivi (es. indennità, premi) di importo superiore rispetto all’aumento del minimo tabellare previsto per il nuovo livello, tali eccedenze possono essere “assorbite” fino a concorrenza dei nuovi minimi tabellari del livello superiore.  
   - Gli **scatti di merito** non sono assorbibili: rimangono invariati e continuano a maturare secondo le norme vigenti.

2. **Compensazione temporanea per man

In [ ]:
test_query("Cosa dice il CCNL sulle assenze?")

==================Routing query to files: ccnl_commercio_terziario_distribuzione_e_servizi.pdf
Filtering by titles: Art.33 – Assenze

Domanda: Cosa dice il CCNL sulle assenze?

Risposta: Il CCNL disciplina le assenze nel seguente modo:

1. **Comunicazione tempestiva** – Qualora si verifichi un grave impedimento, il dipendente è tenuto a informare immediatamente l’azienda, affinché sia possibile organizzare la sostituzione e garantire la continuità dell’attività.

2. **Documentazione** – L’assenza deve essere giustificata entro il giorno successivo all’evento, mediante certificazione o altra documentazione ritenuta idonea.

3. **Conseguenze per l’inosservanza** – Il mancato rispetto di quanto previsto al punto 1 comporta l’applicazione di provvedimenti disciplinari secondo l’art. 49. Inoltre, il datore di lavoro può trattenere, ai sensi dell’art. 24, le quote giornaliere della normale retribuzione corrispondenti alle giornate di assenza ingiustificata.


In [5]:
test_query("Se dovessi assentarmi dal lavoro, posso comunicarlo a voce il giorno stesso?")


Domanda: Se dovessi assentarmi dal lavoro, posso comunicarlo a voce il giorno stesso?

Risposta: Gentile dipendente,

per poter assentarsi dal lavoro è necessario avvertire i propri preposti **prima dell’inizio del normale orario di lavoro** del giorno in cui si verifica l’assenza. La comunicazione può avvenire per telefono, e‑mail o fax (o tramite un’interposta persona), come previsto dall’articolo 34 relativo alla malattia e all’infortunio non sul lavoro.  

Solo in casi di giustificato e documentato impedimento o di forza maggiore, che non consentono la comunicazione preventiva, è ammessa una segnalazione successiva, ma dovrà comunque essere fornita la giustificazione (certificato medico) entro il secondo giorno di assenza.  

Pertanto, una semplice comunicazione verbale **il giorno stesso, dopo l’inizio dell’orario di lavoro**, non è conforme alle disposizioni aziendali, salvo che ricorrano le ipotesi di forza maggiore o impedimento giustificato. In tali situazioni è comunque obbl

### Test File Routing vs Not Routing

In [6]:
test_query("Si può fumare secondo il regolamento aziendale?")

==================Routing query to files: BIS - Regolamento Aziendale.pdf
Filtering by titles: 16. Divieto di fumo e bevande alcoliche

Domanda: Si può fumare secondo il regolamento aziendale?

Risposta: No. Il regolamento aziendale prevede il divieto assoluto di fumo all’interno di tutti i locali aziendali; è consentito fumare esclusivamente all’esterno dei locali.


In [7]:
test_query("Esistono spazi all'interno per fumare?")


Domanda: Esistono spazi all'interno per fumare?

Risposta: Non vi sono spazi destinati al fumo all’interno dei locali aziendali. Il divieto di fumo è assoluto in tutti gli ambienti interni dell’azienda; è consentito fumare esclusivamente all’esterno.


## Out-of-Domain Queries

In [8]:
test_query("Chi era Albert Einstein?")


Domanda: Chi era Albert Einstein?

Risposta: L'argomento della domanda non è pertinente.


In [9]:
test_query("Potresti spiegarmi la differenza tra srl ed spa?")


Domanda: Potresti spiegarmi la differenza tra srl ed spa?

Risposta: L'argomento della domanda non è pertinente.


In [10]:
test_query("Quali sono gli indirizzi IBAN dei conti presenti in tutti i documenti?")


Domanda: Quali sono gli indirizzi IBAN dei conti presenti in tutti i documenti?

Risposta: L'argomento della domanda non è pertinente.
